# Crear un mapa de SLAM

En esta actividad, se creará un mapa de SLAM (simultaneous localization and mapping) de forma automática. Hay que empezar a hacer *mapping* y crear suscripciones a los websockets del IMU (intertial measurement unit) y del sistema de SLAM, que tienen información relevante. Después, hay que hacer que el robot gire para ayudarlo a obtener *pose*. Luego, se hace que el robot se mueva en cruz para que haga un mapa. Finalmente, hay que detener el *mapping* y mostrar el mapa obtenido.

Para contestar la actividad, sustituye los elipses (...) en cada celda con los elementos descritos.

Primero hay que importar los módulos necesarios. El objeto `misty.Robot` permite interactuar con el wrapper, `numpy` sirve para hacer manipulaciones de matrices y `matplotlib` hace visualizaciones con los datos (relevante para el mapa). Se usa el módulo `random` para generar identificadores para las suscripciones a los WebSockets, y `time` permite esperar entre comandos.

In [1]:
from misty import Robot
from random import randrange
import numpy as np
import matplotlib.pyplot as plt
import time

## Crear objeto de `Robot`

Hay que crear una instancia del objeto `Robot`, que permite interactuar con el Misty a través de su API. Su constructor sólo requiere pasar la dirección IP como `str`. Puedes obtener la dirección IP con la aplicación de Misty.

Además, hay que definir una variable `debounce` para usarla como parámetro en las suscripciones a websockets. Representa qué tanto tiempo va a transcurrir entre cada vez que el websocket manda información (en milisegundos).

In [ ]:
alfredo = Robot(...)
debounce = 250

# crear identificadores unicos
slam = 'slam' + str(randrange(0, 999))
imu = 'imu' + str(randrange(0, 999))

""" ---------- solucion ----------
alfredo = Robot('192.168.1.81')
debounce = 250

# crear identificadores unicos
slam = 'slam' + str(randrange(0, 999))
imu = 'imu' + str(randrange(0, 999))
"""

## Comenzar a hacer mapas y crear suscripciones de websockets

Ahora, es necesario indicarle al Misty que comience a hacer un mapa. Primero, hay que asegurarse de que su cabeza esté enderezada para que alcance a "ver" los objetos a su alrededor. En el sistema de coordenadas de Misty, esto corresponde a la posición $(0,0,0)$. 

Luego, hay que asegurarse de que Misty no esté ya haciendo mapping y que los sensores de SLAM estén listos. Para esto, se utiliza el método `stop_mapping` y `reset_slam`. Ahora se puede llamar el método `start_mapping()` de la instancia de `Robot` para comenzar el mapa.

Después, hay que crear objetos que representen a los websockets del status de SLAM y del IMU a través del objeto de `Robot`. Al hacerlo, hay que pasar un parámetro de `event_name`, que es un identificador único de la suscripción que se va a hacer (puede ser cualquier `str`). Estos objetos quedan almacenados en el diccionario de `websockets` del robot, y cada uno puede accesarse a través de su identificador único. Hay que accesar estos objetos y llamar el método `subscribe` para iniciar las suscripciones. Las suscripciones permiten recibir información del Robot que emite desde distintos websockets. En este caso, puesto que se busca hacer un mapa de SLAM, es relevante el status de SLAM y los valores de posición del robot.

In [ ]:
# comenzar a mapear
alfredo.stop_mapping()
print('stopping mapping...')
time.sleep(5)
alfredo....()
print('resetting SLAM...')
time.sleep(10)
alfredo.start_mapping()
print('starting mapping...')

# suscribirse a slam 
alfredo.add_slam_status(slam, debounce=debounce)
alfredo.websockets[...].subscribe()

# suscribirse a imu 
alfredo.add_imu(imu, debounce=debounce)
...

""" ---------- solucion ----------
# comenzar a mapear
alfredo.stop_mapping()
print('stopping mapping...')
time.sleep(5)
alfredo.reset_slam()
print('resetting SLAM...')
time.sleep(10)
alfredo.start_mapping()
print('starting mapping...')

# suscribirse a slam 
alfredo.add_slam_status('slam', debounce=debounce)
alfredo.websockets['slam'].subscribe()

# suscribirse a imu 
alfredo.add_imu('imu', debounce=debounce)
alfredo.websockets['imu'].subscribe()
"""

## Obtener pose

Para que Misty pueda crear mapas, es necesario que tenga *pose*. Esto significa que sepa dónde está dentro del espacio (su orientación, posición, etc.). Dependiendo del espacio en que se encuentre (si hay poco contraste, si hay pocos objetos, si hay poca luz, etc.), puede ser difícil que obtenga pose. El proceso puede facilitarse si se hace que Misty gire lentamente alrededor de su propio eje.

1. Vamos a utilizar el método `drive_time` para hacer que el robot gire una cierta cantidad de tiempo. Toma como argumentos a `linear_velocity`, `angular_velocity` (como porcentajes de las velocidades máximas que logra el robot, entre -100 y 100) y `time_ms` (en ms). Queremos que gire al 20% de su velocidad angular máxima.
1. Se declara una variable que almacena si ya se tiene pose o no.
1. Se comienza un bucle que debe repetirse mientras no se tenga pose y todavía no acabe el tiempo de espera.
    1. Se obtiene un valor del websocket de SLAM, que es un diccionario con varios elementos que representan información del sistema de SLAM (ver documentación). Una forma de ver si el robot tiene pose es a través de la lista que contiene el `statusList` dentro de `slamStatus` del diccionario obtenido; si contiene a la cadena `HasPose`, significa que el robot tiene pose.
    1. Si `statusList` contiene a `HasPose`, entonces hay que cambiar a la variable que almacena si se tiene pose.
1. Cuando se obtenga pose, el robot debe detenerse. La función `stop` hace que se detengan los actuadores de Misty.

In [ ]:
# girar hasta obtener pose
wait_time = 30 # segundos
alfredo.drive_time(0, ..., ... * 1000) # vel angular de 5%
has_pose = ...
start_time = time.time() # obtiene el tiempo en ese instante (en s)

while not has_pose and (time.time() - start_time < ...):
    slamMes = alfredo.websockets[slam].get_message()
    if 'HasPose' in slamMes['slamStatus']['...']:
        has_pose = True
alfredo....()

"""
# girar hasta obtener pose
wait_time = 30 # segundos
alfredo.drive_time(0, 20, wait_time * 1000) # vel angular de 5%
has_pose = False
start_time = time.time() # obtiene el tiempo en ese instante (en s)

while not has_pose and (time.time() - start_time < wait_time):
    slamMes = alfredo.websockets[slam].get_message()
    if 'HasPose' in slamMes['slamStatus']['statusList']:
        has_pose = True
alfredo.stop()
"""

## Desplazar a Misty para crear mapa

En este punto, Misty puede hacer un mapa del espacio en que se encuentra. Para lograrlo, vamos a hacer que Misty se desplace en forma de una cruz y dé una vuelta en cada extremo. Las funciones de movimiento a través del API de REST que utiliza el wrapper de Python son inestables, así que se va a usar un programa de JS llamado `Cross.js`. Para que Misty pueda usarlo, también requiere de un archivo `Cross.json`.

Se manda el conjunto de archivos a Misty dentro de un fichero zip (`Cross.zip`). Se utiliza el método `save_skill` para esto, que toma los siguientes parámetros:
1. `file`: la ruta del zip que se va a mandar
1. `immediately_apply`: indica si el programa debe ejecutarse inmediatamente (en este caso, sí)
1. `overwrite_existing`: si se debe sobreescribir archivos existentes del mismo nombre

In [2]:
# ruta del archivo
path = ...

alfredo.save_skill(..., ..., true)

"""
# ruta del archivo
path = pendiente

alfredo.save_skill(path, true, true)
"""

NameError: name 'alfredo' is not defined

## Mostrar el mapa obtenido

Después de que el robot se haya movido, es posible ver el mapa que hizo. Para eso, se necesita la función `get_map`. Regresa un diccionario con información acerca del mapa. Se van a utilizar tres de los valores que contiene:

- `slam_map['grid']`: Lista que contiene los valores para cada punto sobre la cuadrícula del mapa.
- `slam_map['height']`: La cantidad de hileras en la cuadrícula del mapa.
- `slam_map['width']`: La cantidad de columnas en la cuadrícula del mapa.

Después de obtener el mapa, hay que detener el proceso de mapeo con `stop_mapping`. Esto desactiva algunos sensores, lo que ayuda a que la batería de Misty no se agote tan rápido.

Finalmente, se muestra la cuadrícula del mapa con `matplotlib`.

In [ ]:
# obtener mapa
slam_map = alfredo.get_map()
grid = np.reshape(slam_map['grid'], (..., ...)) # altura, ancho

# dejar de mapear
alfredo....()

# desuscribirse de websockets
alfredo.websockets[slam].unsubscribe()
alfredo.websockets[...].unsubscribe()

# mostrar mapa
plt.pcolormesh(...)
plt.colorbar()
plt.show()

""" ---------- solucion ----------
# obtener mapa
slam_map = alfredo.get_map()
grid = np.reshape(slam_map['grid'], (slam_map['height'], slam_map['width']))

# dejar de mapear
alfredo.stop_mapping()

# desuscribirse de websockets
alfredo.websockets[slam].unsubscribe()
alfredo.websockets[imu].unsubscribe()

# mostrar mapa
plt.pcolormesh(grid)
plt.colorbar()
plt.show()
"""